In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from yellowbrick.cluster import SilhouetteVisualizer

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Create dummy data (3 dimensions)

In [ ]:
data = make_blobs(n_samples=200 , n_features=3, centers=4, cluster_std=0.2, center_box=(-1, 1), random_state=420)
data

In [ ]:
df = pd.DataFrame(data[0], columns=["feature1", "feature2", "feature3"])
df['cluster'] = pd.Categorical(data[1].astype(str), categories=["0", "1", "2", "3"], ordered=True)

df.head()

## Dimensional Visualizations

Inspect your data and consider outlier removal and standardization/normalization.

In [ ]:
fig = make_subplots(rows=1, cols=3)

clusters = list(df["cluster"].unique())
clusters.sort()
colors = px.colors.qualitative.Plotly

# Subplot 1: 2 + 1
for i, c in enumerate(clusters):
    subset = df[df['cluster'] == c]
    fig.add_trace(go.Scatter(
        x=subset["feature1"], 
        y=subset["feature2"], 
        mode='markers',
        name=c,
        legendgroup=c,
        marker=dict(color=colors[i]),
    ), row=1, col=1)
    
    
fig.update_xaxes(title_text="feature1", row=1, col=1, title_standoff=10, showticklabels=False)
fig.update_yaxes(title_text="feature2", row=1, col=1, title_standoff=5, showticklabels=False)

# Subplot 2: 3 + 1
for i, c in enumerate(clusters):
    subset = df[df['cluster'] == c]
    fig.add_trace(go.Scatter(
        x=subset["feature1"], 
        y=subset["feature3"], 
        mode='markers',
        name=c,
        legendgroup=c,
        showlegend=False,
        marker=dict(color=colors[i]),
    ), row=1, col=2)
    
fig.update_xaxes(title_text="feature1", row=1, col=2, title_standoff=10, showticklabels=False)
fig.update_yaxes(title_text="feature3", row=1, col=2, title_standoff=5, showticklabels=False)
    
# Subplot 3: 3 + 2
for i, c in enumerate(clusters):
    subset = df[df['cluster'] == c]
    fig.add_trace(go.Scatter(
        x=subset["feature2"], 
        y=subset["feature3"],  
        mode='markers',
        name=c,
        legendgroup=c,
        showlegend=False,
        marker=dict(color=colors[i]),
    ), row=1, col=3)

fig.update_xaxes(title_text="feature2", row=1, col=3, title_standoff=10, showticklabels=False)
fig.update_yaxes(title_text="feature3", row=1, col=3, title_standoff=5, showticklabels=False)

fig.update_layout(height=400, width=900, title_text="Feature Pairs Colored by Cluster")

fig.show()

In [ ]:
px.scatter_3d(
    df, 
    x="feature1", y="feature2", z="feature3", 
    color="cluster", 
    category_orders={"cluster": ["0", "1", "2", "3"]},
    width=600, height=500
)


## Elbow Method

In [ ]:
inertia = []

for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, random_state=420).fit(df.values)
    inertia.append(kmeans.inertia_)
    
inertia

In [ ]:
px.line(
    x=range(1,10), 
    y=inertia,
    labels={
        "y": "Inertia", 
        "x": "Cluster #"
    },
    width=500, height=300
)

## Silhouette Analysis

[Silhouette Visualizer](https://www.scikit-yb.org/en/latest/api/cluster/silhouette.html) comes from the `yellowbrick` package, which will need to be installed in your Anaconda Environment. 

In [ ]:
k_clusters = 4 # increment to compare different cluster #s
kmeans = KMeans(n_clusters=k_clusters, random_state=420) 
silhouette_avg = silhouette_score(df, kmeans.fit_predict(df.values))
silhouette_avg

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors=colors)
visualizer.fit(df.values)
visualizer.show()

In [ ]:
df["cluster"] = data[1].astype(str)
df['cluster'] = pd.Categorical(df['cluster'], categories=["0", "1", "2", "3"], ordered=True)

In [ ]:
df_preds = df.copy()
df_preds["cluster"] = pd.Categorical(
    kmeans.labels_.astype(str), 
    categories=["0", "1", "2", "3"], 
    ordered=True
)


px.scatter_3d(
    df_preds, 
    x="feature1", y="feature2", z="feature3", 
    height=500, width=600, color="cluster",
    category_orders={"cluster": ["0", "1", "2", "3"]},
    title=f"{k_clusters} Clusters | Silhouette Average: {silhouette_avg}"
)